In [21]:
### https://adventofcode.com/2023/day/1


import re



TXT_2_DIGIT = {
    "one": "1", 
    "two": "2", 
    "three": "3", 
    "four": "4", 
    "five": "5", 
    "six": "6", 
    "seven": "7", 
    "eight": "8", 
    "nine": "9"
}


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    digits = list(filter(None, [''.join(re.findall(r'\d+', item)) for item in data]))
    return sum([int(item[:1] + item[-1]) for item in digits])
        
    
def get_solution_2(data):
    data_preprocessed = []    
    for item in data:
        for key, digit in TXT_2_DIGIT.items():
            item = item.replace(key, key[:-1] + str(digit) + key[1:])
        data_preprocessed.append(item)
    return get_solution_1(data_preprocessed)




file = "data/day_1.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Solution 1-1:", solution_1)
print("Solution 1-2:", solution_2) 

Solution 1-1: 54667
Solution 1-2: 54203


In [19]:
### https://adventofcode.com/2023/day/2


import numpy as np

CUBES_MAX_NUM = {
    "blue": 14, 
    "green": 13, 
    "red": 12
}


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return filter(None, [line.strip() for line in f_in.readlines()])
    

    
def get_games_dict(data):
    games_dict = dict()    
    for line in data:
        game_id, game_sets = line.strip().split(':')
        game_id = game_id.replace('Game ', '').strip()
        game_sets = game_sets.strip().split(';')
        game_set_dict = dict()
        for game_set in game_sets:            
            game_set = game_set.strip().split(',')
            for item in game_set:
                num_cubes, cube_color = item.strip().split(' ')
                if not cube_color in game_set_dict:
                    game_set_dict[cube_color] = [int(num_cubes)]
                else:
                    game_set_dict[cube_color] += [int(num_cubes)]
        games_dict[game_id] = game_set_dict
    return games_dict


def is_valid_game_set(max_colors_dict):
    for color, max_value in max_colors_dict.items():
        if max_value > CUBES_MAX_NUM[color]:
            return False
    return True


def get_solution_1(data):
    valid_games = []
    for game_id, game_sets in data.items():
        max_colors = dict()
        for color, num_cubes_list in game_sets.items():
            if not num_cubes_list:
                num_cubes_list = [0]
            max_colors[color] = max(num_cubes_list)
        if is_valid_game_set(max_colors):
            valid_games.append(int(game_id))
    return sum(valid_games)
            
        
    
def get_solution_2(data):
    power_sum = 0
    for game_id, game_sets in data.items():
        max_colors = []
        for color, num_cubes_list in game_sets.items():
            if not num_cubes_list:
                num_cubes_list = [0]
            max_colors.append(max(num_cubes_list))
        power_sum += np.prod(max_colors)
    return power_sum
            



file = "data/day_2.txt"

data = get_games_dict(read_file(file))
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 2 - Solution 1:", solution_1)
print("Day 2 - Solution 2:", solution_2)

Day 2 - Solution 1: 2683
Day 2 - Solution 2: 49710


In [17]:
### https://adventofcode.com/2023/day/3


import re
import numpy as np



def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines() if line.strip()]
    

def is_symbol(char):
    return not bool(re.match(r"[A-Za-z0-9.]+", char))


def has_adjacent_symbol(matrix, i_row, i_col, num_rows, num_cols, current_number):    
    left = is_symbol(matrix[i_row, i_col - 1]) if i_col > 0 else False
    right = is_symbol(matrix[i_row, i_col + 1]) if i_col < num_cols - 1 else False
    top, bottom = False, False
    start = i_col - 1 if i_col > 0 else 0
    end = i_col + 2 if i_col < num_cols else num_cols - 1
    if i_row > 0:        
        top_region = matrix[i_row - 1, start : end]
        top = any([is_symbol(item) for item in top_region])
    if i_row < num_rows - 1:
        bottom_region = matrix[i_row + 1, start : end]
        bottom = any([is_symbol(item) for item in bottom_region])
    return any([left, right, top, bottom])
    

def get_solution_1(data):
    part_numbers = []
    matrix = np.array([list(line) for line in data])
    num_rows, num_cols = matrix.shape
    for idx_row, row in enumerate(matrix):
        current_number = ""
        is_adjacent_to_symbol = False
        for idx_col, cell_item in enumerate(row):
            if cell_item.isnumeric():
                current_number += cell_item 
                if not is_adjacent_to_symbol:
                    is_adjacent_to_symbol = has_adjacent_symbol(matrix, idx_row, idx_col, num_rows, num_cols, current_number)
                if idx_col < num_cols - 1 and not matrix[idx_row, idx_col + 1].isnumeric() or \
                idx_col == num_cols - 1:
                    if is_adjacent_to_symbol:
                        part_numbers.append(int(current_number))
                    current_number = ""
                    is_adjacent_to_symbol = False
    return sum(part_numbers)


def get_digits_by_side_shift(row, idx, num_cols, shift):
    number = ""
    while row[idx].isnumeric():
        number += row[idx]
        idx += shift
        if idx < 0 or idx == num_cols:
            number = number[::-1] if shift == -1 else number
            return number
    number = number[::-1] if shift == -1 else number
    return number


def get_number(row, current_idx, num_cols):
    current_val = row[current_idx]
    if current_val.isnumeric():
        return int(get_digits_by_side_shift(row, current_idx - 1, num_cols, shift=-1) + 
                   current_val + 
                   get_digits_by_side_shift(row, current_idx + 1, num_cols, shift=1))
    return None


def get_gear_region_value(row, region, start, end, num_cols):
    check_multi_region = list(filter(None, re.split(r"\D+", ''.join(region))))
    if len(check_multi_region) == 2:
        i_1 = start
        i_2 = end - 1
        res_1 = get_number(row, i_1, num_cols)
        res_2 = get_number(row, i_2, num_cols)
        if res_1 and res_2:            
            return [res_1, res_2]
        return [None]

    for i, char in enumerate(region):        
        if char.isnumeric():
            return [get_number(row, start + i, num_cols)]
    return [None]


def contains_digits(region):
    return bool(re.match(r"\D*\d+\D*", ''.join(region)))


def get_adjacent_numbers(matrix, i_row, i_col, num_rows, num_cols):
    items = []
    # left
    items += [get_number(matrix[i_row], i_col - 1, num_cols)] if i_col > 0 else [None]
    # right
    items += [get_number(matrix[i_row], i_col + 1, num_cols)] if i_col < num_cols - 1 else [None]
    top, bottom = None, None
    start = i_col - 1 if i_col > 0 else 0
    end = i_col + 2 if i_col < num_cols else num_cols - 1
    if i_row > 0:        
        top_region = matrix[i_row - 1, start : end]
        # top
        items += get_gear_region_value(matrix[i_row - 1], top_region, start, end, num_cols) if contains_digits(top_region) else [None]        
    if i_row < num_rows - 1:
        bottom_region = matrix[i_row + 1, start : end]
        # bottom
        test = False
        if i_row ==132:
            test = True
        items += get_gear_region_value(matrix[i_row + 1], bottom_region, start, end, num_cols) if contains_digits(bottom_region) else [None]
#     print(matrix[i_row, i_col],i_row, i_col)
#     print(items)
    items = [item for item in items if item]
#     print(items)
    if len(items) != 2:
#         print("-> invalid, is not gear")
#         print("--------------------------------------")
        return 0
#     print(items)
#     print("Valid, result of multiplication: ",np.prod(items))
#     print("--------------------------------------")
    return np.prod(items)

    
def get_solution_2(data):
    gear_numbers = 0
    matrix = np.array([list(line) for line in data])
    num_rows, num_cols = matrix.shape
    for idx_row, row in enumerate(matrix):
        for idx_col, cell_item in enumerate(row):
            if cell_item == '*':
                adj_number = get_adjacent_numbers(matrix, idx_row, idx_col, num_rows, num_cols)
                gear_numbers += adj_number
    return gear_numbers



file = "data/day_3.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 3 - Solution 1:", solution_1)
print("Day 3 - Solution 2:", solution_2)

Day 3 - Solution 1: 546563
Day 3 - Solution 2: 91031374


In [ ]:
### https://adventofcode.com/2023/day/4


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_4.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 4 - Solution 1:", solution_1)
print("Day 4 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/5


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_5.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 5 - Solution 1:", solution_1)
print("Day 5 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/6


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_6.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 6 - Solution 1:", solution_1)
print("Day 6 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/7


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_7.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 7 - Solution 1:", solution_1)
print("Day 7 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/8


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_8.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 8 - Solution 1:", solution_1)
print("Day 8 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/9


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_9.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 9 - Solution 1:", solution_1)
print("Day 9 - Solution 2:", solution_2)

In [ ]:
### https://adventofcode.com/2023/day/10


def read_file(file_name):
    with open(file_name, 'r') as f_in:
        return [line.strip() for line in f_in.readlines()]
    

def get_solution_1(data):
    pass
        
    
def get_solution_2(data):
    pass




file = "data/day_10.txt"

data = read_file(file)
solution_1 = get_solution_1(data)
solution_2 = get_solution_2(data)

print("Day 10 - Solution 1:", solution_1)
print("Day 10 - Solution 2:", solution_2)